In [38]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
import FinanceDataReader as fdr
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [39]:
def preprocess_(path,  column_, slice='',  encoding=None, header='infer'):

    df = pd.read_csv(path, encoding=encoding, header=header)
    
    if slice:
        df = df[slice]

    df.columns = column_

    if (type(df.iloc[:,1][1])==str):
        df.iloc[:,1] = df.iloc[:,1].apply(lambda x: float(x.replace(',', '')))
        
    df['Date'] = df['Date'].str.replace('[^0-9]', '')
    df['Date'] = df['Date'].apply(lambda x: x[:4]+'-'+x[4:6]+'-'+x[6:])
    df['Date'] = pd.to_datetime(df['Date'])

    df.sort_values('Date', inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'], axis=1, inplace=True)

    return df

In [40]:
dt = pd.read_csv("./datasets/merge.csv")
dt['Date'] = pd.to_datetime(dt['Date'])
ks = preprocess_(path="./datasets/KS200.csv", slice=['Date', 'Close'], column_=['Date', 'KS200'])
all = pd.merge(ks, dt, how='inner', on='Date')
# data.to_csv("data.csv", index=None)

C:\Users\dbswo\AppData\Local\Temp\ipykernel_6192\2556062775.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Date'] = df['Date'].str.replace('[^0-9]', '')


In [5]:
# data = pd.read_csv("./datasets/data.csv")
# data['Date'] = pd.to_datetime(all['Date'])
# gold = preprocess_(path='./datasets/gold.csv', column_=['Date', 'Gold'], encoding='CP949')
# all = pd.merge(data, gold, how='inner', on='Date')
# all.to_csv("all.csv", index=None)

C:\Users\dbswo\AppData\Local\Temp\ipykernel_18272\2556062775.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Date'] = df['Date'].str.replace('[^0-9]', '')


In [44]:
# all = pd.read_csv('all.csv')
all.columns = ['Date', 'KS200', 'Gold', 'REMX', 'AAPL', 'TSMC', 'ER']

In [41]:
data = all.drop(['Date', 'KS200'], axis=1)
target = all['KS200']

len_ = round(len(data)*0.8)
x_train, x_test = data[:len_], data[len_:]
y_train, y_test = target[:len_], target[len_:]

lr = LinearRegression()
lr.fit(x_train, y_train)
pred = lr.predict(x_test)

mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)
print(f'input_data with none\ntarget with none \nmse : {mse} \nrmse : {rmse} \nr2 : {r2}')

input_data with none
target with none 
mse : 387.3385775266299 
rmse : 19.6809191230143 
r2 : -0.4363341402212313


In [47]:
all

,Date,KS200,Gold,REMX,AAPL,TSMC,ER
0,2020-05-04,250.60,1748.5,29.89,73.29,52.15,1224.14
1,2020-05-06,255.00,1729.2,31.07,75.16,52.13,1228.55
2,2020-05-07,254.46,1768.6,31.08,75.93,52.33,1219.61
3,2020-05-11,254.95,1741.9,31.96,78.75,52.40,1222.91
4,2020-05-12,253.37,1749.2,31.14,77.85,51.10,1226.64
...,...,...,...,...,...,...,...
373,2022-03-25,362.06,1959.8,116.80,174.72,106.73,1225.08
374,2022-03-28,362.06,1944.7,114.81,175.60,106.33,1223.65
375,2022-03-29,363.77,1918.0,117.54,178.96,109.25,1207.35
376,2022-03-30,364.50,1939.0,117.90,177.77,106.73,1210.31


In [48]:
import statsmodels.formula.api as sm
#lm = sm.ols('KS200 ~ Gold_x + REMX + AAPL + TSMC + ER + Gold_y', data=all)
lm = sm.ols('KS200 ~ Gold + REMX + AAPL + TSMC + ER ', data=all)  
res = lm.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  KS200   R-squared:                       0.920
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     857.3
Date:                Tue, 26 Apr 2022   Prob (F-statistic):          1.15e-201
Time:                        14:51:07   Log-Likelihood:                -1559.9
No. Observations:                 378   AIC:                             3132.
Df Residuals:                     372   BIC:                             3155.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    874.2233     59.809     14.617      0.000     756.616     991.830
Gold          -0.0378      0.012     -3.062      0.002      -0.062      -0.014
REMX           0.9663      0.086     11.206      0.000       0.797       1.136
AAPL          -0.6572      0.095     -6.890      0.000      -0.845      -0.470
TSMC           1.2868      0.138      9.320      0.000       1.015       1.558
ER            -0.4769      0.043    -11.042      0.000      -0.562      -0.392
==============================================================================
Omnibus:                        7.906   Durbin-Watson:                   0.156
Prob(Omnibus):                  0.019   Jarque-Bera (JB):                5.191
Skew:                          -0.129   Prob(JB):                       0.0746
Kurtosis:                       2.487   Cond. No.                     1.69e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.69e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [53]:
data = all.drop(['Date', 'KS200'], axis=1)
target = all['KS200']

len_ = round(len(data)*0.8)
x_train, x_test = data[:len_], data[len_:]
y_train, y_test = target[:len_], target[len_:]

# input_data에 log, 차분 적용
data_log = np.log1p(data)
data_diff = data_log.diff(1)
data_diff.dropna(inplace=True)
data_diff.reset_index(inplace=True)
data_diff.drop(['index'], axis=1, inplace=True)

# input_data의 줄어든 길이만큼 target 길이 조정
target = target[1:]
target = pd.DataFrame(target)
target.reset_index(inplace=True)
target.drop(['index'], axis=1, inplace=True)
target = np.log1p(target)
target = StandardScaler().fit_transform(target)

# 데이터 순서대로 80% 훈련데이터 20% 검증데이터 설정
len_ = round(len(data)*0.8)
x_train, x_test = data_diff[:len_], data_diff[len_:]
y_train, y_test = target[:len_], target[len_:]

lr = LinearRegression()
lr.fit(x_train, y_train)
pred = lr.predict(x_test)

# pred_ex = np.expm1(pred)
# y_test_ex = np.expm1(y_test)

# mse = mean_squared_error(y_test_ex, pred_ex)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test_ex, pred_ex)
# print(f'input_data with log, diff \ntarget with log, standardscaler, expm1\nmse : {mse} \nrmse : {rmse} \nr2 : {r2}')

mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)
print(f'input_data with log, diff \ntarget with log \nmse : {mse} \nrmse : {rmse} \nr2 : {r2}')

input_data with log, diff 
target with log 
mse : 0.12253473904533792 
rmse : 0.35004962368975334 
r2 : -0.4667833618567594


In [54]:
data = all.drop(['Date', 'KS200'], axis=1)
target = all['KS200']

len_ = round(len(data)*0.8)
x_train, x_test = data[:len_], data[len_:]
y_train, y_test = target[:len_], target[len_:]

# input_data에 log, 차분 적용
data_log = np.log1p(data)
data_diff = data_log.diff(1)
data_diff.dropna(inplace=True)
data_diff.reset_index(inplace=True)
data_diff.drop(['index'], axis=1, inplace=True)
# data_diff = data_diff['AAPL']
# data_diff = data_diff.array.reshape(-1, 1)

# input_data의 줄어든 길이만큼 target 길이 조정
target = target[1:]
target = pd.DataFrame(target)
target.reset_index(inplace=True)
target.drop(['index'], axis=1, inplace=True)
# target = np.log1p(target)
# target = StandardScaler().fit_transform(target)

# 데이터 순서대로 80% 훈련데이터 20% 검증데이터 설정
len_ = round(len(data)*0.8)
x_train, x_test = data[:len_], data[len_:]
y_train, y_test = target[:len_], target[len_:]

lr = LinearRegression()
lr.fit(x_train, y_train)
pred = lr.predict(x_test)

# pred_ex = np.expm1(pred)
# y_test_ex = np.expm1(y_test)

# mse = mean_squared_error(y_test_ex, pred_ex)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test_ex, pred_ex)
# print(f'input_data with log, diff \ntarget with log, standardscaler, expm1\nmse : {mse} \nrmse : {rmse} \nr2 : {r2}')

mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)
print(f'input_data with log, diff \ntarget with log \nmse : {mse} \nrmse : {rmse} \nr2 : {r2}')

ValueError: Found input variables with inconsistent numbers of samples: [75, 76]

In [ ]:
data = all.drop(['Date', 'KS200'], axis=1)
target = all['KS200']

len_ = round(len(data)*0.8)
x_train, x_test = data[:len_], data[len_:]
y_train, y_test = target[:len_], target[len_:]

# # input_data에 log, 차분 적용
# data_log = np.log1p(data)
# data_diff = data_log.diff(1)
# data_diff.dropna(inplace=True)
# data_diff.reset_index(inplace=True)
# data_diff.drop(['index'], axis=1, inplace=True)
# data_diff = data_diff['AAPL']
# data_diff = data_diff.array.reshape(-1, 1)

# # input_data의 줄어든 길이만큼 target 길이 조정
# target = target[1:]
# target = pd.DataFrame(target)
# target.reset_index(inplace=True)
# target.drop(['index'], axis=1, inplace=True)
# target = np.log1p(target)
# target = StandardScaler().fit_transform(target)

# 데이터 순서대로 80% 훈련데이터 20% 검증데이터 설정
len_ = round(len(data)*0.8)
x_train, x_test = data[:len_], data[len_:]
y_train, y_test = target[:len_], target[len_:]

lr = LinearRegression()
lr.fit(x_train, y_train)
pred = lr.predict(x_test)

# pred_ex = np.expm1(pred)
# y_test_ex = np.expm1(y_test)

# mse = mean_squared_error(y_test_ex, pred_ex)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test_ex, pred_ex)
# print(f'input_data with log, diff \ntarget with log, standardscaler, expm1\nmse : {mse} \nrmse : {rmse} \nr2 : {r2}')

mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, pred)
print(f'input_data with log, diff \ntarget with log \nmse : {mse} \nrmse : {rmse} \nr2 : {r2}')